In [4]:
import pandas as pd
import numpy as np

In [5]:
# import xslx file
url = r'C:\Users\rergu\OneDrive\IronHack\MiniProjects\Week3-Project\tfl-daily-cycle-hires.xlsx'
df_bike = pd.read_excel(url, sheet_name='Data')
df_bike.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
df_bike.head()

,Day,Number of Bicycle Hires
0,2015-01-01,9615
1,2015-01-02,15389
2,2015-01-03,5779
3,2015-01-04,9367
4,2015-01-05,20566


In [4]:
df_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Day                      1827 non-null   datetime64[ns]
 1   Number of Bicycle Hires  1827 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 28.7 KB


In [7]:
#rename the columns
df_bike.columns = [col.lower() for col in df_bike.columns]
df_bike.columns = [col.replace(" ", "_") for col in df_bike.columns]
df_bike.rename(columns= {
    'day': 'date'
}, inplace=True)

In [8]:
# get day, month and year from date
df_bike['day'] = df_bike['date'].dt.day
df_bike['month'] = df_bike['date'].dt.month
df_bike['year'] = df_bike['date'].dt.year

In [9]:
# group by month and day
grouped_df_bike = df_bike.groupby(['month', 'day']).agg(
    number_of_bicycle_hires_mean=('number_of_bicycle_hires', 'mean'),
    number_of_bicycle_hires_std=('number_of_bicycle_hires', 'std')
).reset_index()


In [ ]:
#index of 
#grouped_df_bike = grouped_df_bike.set_index(['month', 'day'])

In [10]:
# group by year, month and day
grouped_year_df_bike = df_bike.groupby(['year', 'month', 'day']).agg(
    number_of_bicycle_hires_sum=('number_of_bicycle_hires', 'sum')
).reset_index()



In [12]:
# drop feb 29
condition1 = grouped_year_df_bike['month'] == 2
condition2 = grouped_year_df_bike['day'] == 29

index_feb29 = grouped_year_df_bike[condition1 & condition2].index

grouped_year_df_bike.drop(index_feb29, inplace=True)
grouped_df_bike.dropna(subset='bicycle_hires_std', inplace=True)

grouped_df_bike.reset_index(drop=True, inplace=True)
grouped_year_df_bike.reset_index(drop=True, inplace=True)

In [14]:
# data cleaning and finetuning 
grouped_year_df_bike.rename(columns=({'number_of_bicycle_hires_sum': 'bicycle_hires'}), inplace=True)
grouped_df_bike.rename(columns=({'number_of_bicycle_hires_mean': 'bicycle_hires_ave', 'number_of_bicycle_hires_std': 'bicycle_hires_std'}), inplace=True)
grouped_df_bike['bicycle_hires_ave'] = grouped_df_bike['bicycle_hires_ave'].apply(round)


grouped_df_bike['bicycle_hires_std'] = grouped_df_bike['bicycle_hires_std'].apply(round)

In [13]:
# get the index of na row -->
grouped_df_bike.loc[pd.isna(grouped_df_bike["bicycle_hires_std"]), :].index

Index([], dtype='int64')

In [13]:
grouped_year_df_bike.head(10)

,year,month,day,bicycle_hires
0,2015,1,1,9615
1,2015,1,2,15389
2,2015,1,3,5779
3,2015,1,4,9367
4,2015,1,5,20566
5,2015,1,6,20765
6,2015,1,7,22504
7,2015,1,8,15769
8,2015,1,9,22285
9,2015,1,10,14877


In [14]:
grouped_df_bike.head(10)

,month,day,bicycle_hires_ave,bicycle_hires_std
0,1,1,10268,2463
1,1,2,13746,4598
2,1,3,14422,8329
3,1,4,18108,5169
4,1,5,20549,3888
5,1,6,18224,4062
6,1,7,18507,5652
7,1,8,20271,5748
8,1,9,24650,9447
9,1,10,22048,6785


In [15]:
#merge the two dataframes
merged_df_bike = pd.merge(grouped_year_df_bike, grouped_df_bike, on=['month', 'day'], how='left')

In [16]:
#calculate Z-score
"""Compute Z-Scores (Standardized Deviation)
Z-score tells you how many standard deviations away a particular day's value is from the mean:
Z-score = (actual_value - mean)/std

Interpretations -->
z ≈ 0: The day is typical (very close to average).
|z| > 1: The day is somewhat unusual.
|z| > 2: The day is statistically significant — quite rare (could indicate an event, weather impact, etc.).
z < 0: Fewer hires than usual.
z > 0: More hires than usual.
"""

merged_df_bike["z_score"] = (merged_df_bike["bicycle_hires"] - merged_df_bike["bicycle_hires_ave"]
) / merged_df_bike["bicycle_hires_std"]

# Z-score interpretations

def z_score_interpret(z_score):
    if z_score == 0:
        return 'The day is typical'
    elif abs(z_score) > 1:
        return 'The day is unusual'
    elif abs(z_score) > 2:
        return 'The day is significantly unusual'
    elif z_score < 0:
        return 'Fewer hires than usual'
    elif z_score > 0:
        return 'More hires than usual'
    else:
        return 'Nan'

# call it with function 

merged_df_bike['z_score_interpretation'] = merged_df_bike['z_score'].apply(z_score_interpret)

In [17]:
# calculate deviation from the average in percentage and round it 2 decimal places
merged_df_bike['deviation_from_average_pct'] = round(((merged_df_bike['bicycle_hires'] - merged_df_bike['bicycle_hires_ave']) / merged_df_bike['bicycle_hires_ave'] * 100),2)

In [18]:
merged_df_bike.sample(10)

,year,month,day,bicycle_hires,bicycle_hires_ave,bicycle_hires_std,z_score,z_score_interpretation,deviation_from_average_pct
1649,2019,7,9,41447,45974,15440,-0.293199,Fewer hires than usual,-9.85
1508,2019,2,18,23815,22553,2631,0.479666,More hires than usual,5.60
246,2015,9,4,29913,31408,3706,-0.403400,Fewer hires than usual,-4.76
1662,2019,7,22,40842,33890,9915,0.701160,More hires than usual,20.51
1025,2017,10,23,30604,30321,4047,0.069928,More hires than usual,0.93
940,2017,7,30,31893,36043,7077,-0.586407,Fewer hires than usual,-11.51
316,2015,11,13,25410,27138,5009,-0.344979,Fewer hires than usual,-6.37
841,2017,4,22,28323,31049,6956,-0.391892,Fewer hires than usual,-8.78
1455,2018,12,27,12310,9993,2759,0.839797,More hires than usual,23.19
1660,2019,7,20,31351,37222,5760,-1.019271,The day is unusual,-15.77


In [19]:
merged_df_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        1826 non-null   int32  
 1   month                       1826 non-null   int32  
 2   day                         1826 non-null   int32  
 3   bicycle_hires               1826 non-null   int64  
 4   bicycle_hires_ave           1826 non-null   int64  
 5   bicycle_hires_std           1826 non-null   int64  
 6   z_score                     1826 non-null   float64
 7   z_score_interpretation      1826 non-null   object 
 8   deviation_from_average_pct  1826 non-null   float64
dtypes: float64(2), int32(3), int64(3), object(1)
memory usage: 107.1+ KB


In [21]:
# export xlsx file
merged_df_bike.to_excel(r'C:\Users\rergu\OneDrive\IronHack\MiniProjects\Week3-Project\final_df_bike_hires.xlsx', index=False)

